# Считаем ембединги

Настройка colab

In [ ]:
!nvidia-smi

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Библиотеки

In [ ]:
import os
#from google.colab import drive
from tqdm import tqdm
import pickle

import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer, util

## Загрузка файлов

Проверям папку

In [ ]:
comments_dir = "/content/drive/My Drive/comments_clean"
os.listdir(comments_dir)[:5]

['JoeBiden_comments_2020_clean.csv', 'Trump_comments_2020_clean.csv']

загружаем файлы

In [ ]:
%%time
dfs = dict()

for my_file in tqdm(os.listdir(comments_dir), position=0, leave=True):
    with open(os.path.join(comments_dir, my_file), encoding="utf8") as csv_file:
        dfs[my_file] = pd.read_csv(csv_file)

100%|██████████| 2/2 [00:03<00:00,  1.72s/it]

Wall time: 3.44 s


In [ ]:
# add part column with trump
dfs['Trump_comments_2020_clean.csv']['who'] = np.nan
dfs['Trump_comments_2020_clean.csv']['who'] = dfs['Trump_comments_2020_clean.csv']['who'].fillna(0)

# add part column with biden
dfs['JoeBiden_comments_2020_clean.csv']['who'] = np.nan
dfs['JoeBiden_comments_2020_clean.csv']['who'] = dfs['JoeBiden_comments_2020_clean.csv']['who'].fillna(1)

Сразу готовим трейн 100к и тест 20к

In [ ]:
df_train = pd.concat([dfs['Trump_comments_2020_clean.csv'].sample(50000), dfs['JoeBiden_comments_2020_clean.csv'].sample(50000)]) \
                    .drop('Unnamed: 0', axis=1) \
                    .reset_index(drop=True)
df_test = pd.concat([dfs['Trump_comments_2020_clean.csv'].sample(10000), dfs['JoeBiden_comments_2020_clean.csv'].sample(10000)]) \
                    .drop('Unnamed: 0', axis=1) \
                    .reset_index(drop=True)
df_test.head(2)

,author,body,created_utc,link_id,parent_id,score,who
0,CannabisBarbiie,All interviews by FBI agents have 302s. They w...,1584400051,t3_fj4yi3,t1_fkouh5q,1,0.0
1,I_value_my_shit_more,The old is going be torn down to build anew. I...,1592057229,t3_h7trm3,t1_fupasqd,1,0.0


In [ ]:
del dfs

# Векторизация колмментариев

загружаем предобученную модель под сберта

In [ ]:
#model = SentenceTransformer('bert-base-nli-mean-tokens') #эта быстрее
model = SentenceTransformer('bert-large-nli-mean-tokens') #эта точнее
model.max_seq_length = 200

считаем embeddings к комментариям и доаблвяем в таблмцы

In [ ]:
%%time
comment_embedding_train = model.encode(df_train['body'].to_list(), show_progress_bar=True, convert_to_numpy=True, device='cuda')
comment_embedding_test = model.encode(df_test['body'].to_list(), show_progress_bar=True, convert_to_numpy=True, device='cuda')

df_train['embedding'] = list(comment_embedding_train)
df_test['embedding'] = list(comment_embedding_test)


Wall time: 9min 15s


In [ ]:
df_train

,author,body,created_utc,link_id,parent_id,score,who,embedding
0,kkkleina,"does not load for me either, iceland",1606337819,t3_k0zg1t,t1_gdl57gw,1,0.0,"[-0.5620464, -0.0044380575, 0.57985806, 0.1312..."
1,Mine_Darkness08,I swear if they try to twist this into orange ...,1579564912,t3_erfudx,t3_erfudx,1,0.0,"[0.52616477, 0.26897562, 0.55072683, 0.7936976..."
2,Son-of-Neleus,Probably joined the Black Panthers too,1601764899,t3_j42hw9,t1_g7i9o8c,1,0.0,"[1.1628835, -0.5586643, 0.5151502, 0.40808493,..."
3,Shelter-in-Space,"Holy shit this is dumb. 1) ""Dominion"" is just ...",1605737574,t3_jwpm04,t3_jwpm04,-23,0.0,"[0.26338562, -0.5902791, 0.5505226, -0.1255931..."
4,handpeople,He also said Big Mike did not have a dick and ...,1579197712,t3_epmwb6,t3_epmwb6,1,0.0,"[0.7837271, 0.46787333, 0.6279784, -0.34672, 0..."
...,...,...,...,...,...,...,...,...
99995,nevertulsi,I do not think her AG record is trash. I think...,1591991340,t3_h7qb28,t1_fumjk0x,1,1.0,"[0.6386456, -0.51033837, 0.09939509, 0.4628307..."
99996,YourUndoing,My dad has literally (without sarcasm or tryin...,1607985339,t3_kd12mo,t1_gftyph9,1,1.0,"[0.6974588, 0.39496022, 0.46409717, -0.7663132..."
99997,R3dRuby,Every older person in my family despises Trump...,1599674858,t3_iph1sz,t1_g4k3diy,1,1.0,"[0.31154042, -0.33243722, 0.62614083, 0.370053..."
99998,Ode_to_bees,This breaks my heart. I am in NJ and we went t...,1596476798,t3_i2z83p,t3_i2z83p,1,1.0,"[0.6527439, 0.70839363, 0.73180544, 0.44795087..."


Сохраняем полученные таблицы

In [ ]:
with open(r"\Users\Home\Documents\Data Science\comptech\weights\sbert\df_train.pickle", 'wb') as f:
    pickle.dump(df_train, f)

with open(r"\Users\Home\Documents\Data Science\comptech\weights\sbert\df_test.pickle", 'wb') as f:
    pickle.dump(df_test, f)